In [38]:
import spacy
from spacy import Language
from collections import defaultdict
import operator
import math
from string import punctuation, ascii_letters, ascii_uppercase, ascii_lowercase, digits
import random
from itertools import *
import numpy as np
import os
import matplotlib.pyplot as plt
from numpy import asarray
from numpy import exp
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed
import errno
from matplotlib import pyplot
import re
from Levenshtein import distance as levenshtein_distance
from sklearn.metrics import accuracy_score, confusion_matrix
#!python -m spacy info
#!pip install spacy==3.0.5
#!pip install spacy==3.0.3
#!pip install sklearn
from zxcvbn import zxcvbn
import pickle

In [39]:
def mkdir_p(path):
    """To make a directory given a path."""
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
            
def unpack_data(res_folder=None):
    g = []
    br = False
    files = os.listdir(res_folder)
    for file_name in files:
        print(file_name)
        file_path = os.path.join(res_folder, file_name)
        h = pickle.load(open(file_path, 'rb'))
        g.append(h)
        if br:
            break
    
    print('Read Disk')
    print('{} TEST RUNS FOUND'.format(len(g)))

    return g

def word_shape(text=None):
    if len(text) >= 100:
        return "LONG"
    shape = []
    last = ""
    shape_char = ""
    seq = 0
    for char in text:
        if char.isalpha():
            if char.isupper():
                shape_char = "X"
            else:
                shape_char = "x"
        elif char.isdigit():
            shape_char = "d"
        else:
            shape_char = char
        if shape_char == last:
            seq += 1
        else:
            seq = 0
            last = shape_char
        if seq < 4:
            shape.append(shape_char)
    return "".join(shape)

In [40]:
plot_folder = "Results/Simulated_Annealing_Results/"
mkdir_p(plot_folder)

res_folder = "Annealing_Results/"

g = unpack_data(res_folder)
    

20210719_a_123_1_Missing_CHARS_10000_iterations_Passwords_Simulated_Annealing_Extraction.pickle
20210719_a_123_2_Missing_CHARS_10000_iterations_Passwords_Simulated_Annealing_Extraction.pickle
20210719_a_123_3_Missing_CHARS_10000_iterations_Passwords_Simulated_Annealing_Extraction.pickle
20210719_a_123_4_Missing_CHARS_10000_iterations_Passwords_Simulated_Annealing_Extraction.pickle
Read Disk
4 TEST RUNS FOUND


In [42]:
extracted_at_iteration = {1:{},2:{},3:{},4:{}}
accuracy_at_iteration = {1:{},2:{},3:{},4:{}}

max_iterations = 10000
for i in range(1,5):
    iterations = 100
    while iterations<=max_iterations:
        extracted_at_iteration[i][iterations] = []
        accuracy_at_iteration[i][iterations] = []
        iterations = iterations*2


#print(extracted_at_iteration)

for exp in g:
    missing_chars = exp[1]
    extracted_at_iteration_per_password = exp[9]
    secrets = [list(d.keys())[0] for d in extracted_at_iteration_per_password]
    for d in extracted_at_iteration_per_password:
        secret = list(d.keys())[0]
        for iterations in d[secret]:
            extracted_at_iteration[missing_chars][iterations].append(d[secret][iterations])
            #print(iterations, d[secret][iterations])
      
    for iterations in extracted_at_iteration[missing_chars]:
        extracted = extracted_at_iteration[missing_chars][iterations]

        secrets_shape = [word_shape(secret) for secret in secrets]
        extracted_shape = [word_shape(secret) for secret in extracted]

        accuracy = accuracy_score(secrets, extracted)
        shape_accuracy = accuracy_score(secrets_shape, extracted_shape)

        accuracy_at_iteration[missing_chars][iterations].append(accuracy)
        accuracy_at_iteration[missing_chars][iterations].append(shape_accuracy)

#         print("Accuracy = {}".format(accuracy))
#         print("Shape Accuracy = {}".format(shape_accuracy))

    

In [43]:
accuracy_at_iteration

{1: {100: [0.78125, 0.9375],
  200: [0.78125, 0.9375],
  400: [0.78125, 0.9375],
  800: [0.78125, 0.9375],
  1600: [0.78125, 0.9375],
  3200: [0.78125, 0.9375],
  6400: [0.78125, 0.9375]},
 2: {100: [0.02564102564102564, 0.6153846153846154],
  200: [0.1282051282051282, 0.6666666666666666],
  400: [0.07692307692307693, 0.5897435897435898],
  800: [0.1794871794871795, 0.7692307692307693],
  1600: [0.3333333333333333, 0.7948717948717948],
  3200: [0.358974358974359, 0.7435897435897436],
  6400: [0.3333333333333333, 0.717948717948718]},
 3: {100: [0.0, 0.15],
  200: [0.025, 0.275],
  400: [0.0, 0.2],
  800: [0.0, 0.25],
  1600: [0.025, 0.325],
  3200: [0.0, 0.4],
  6400: [0.0, 0.325]},
 4: {100: [0.0, 0.05],
  200: [0.0, 0.05],
  400: [0.0, 0.125],
  800: [0.0, 0.1],
  1600: [0.0, 0.15],
  3200: [0.0, 0.2],
  6400: [0.0, 0.15]}}

In [46]:
print(" ", "1", "2", "3", "4")
for it in [100, 200, 400, 800, 1600, 3200, 6400]:
    print(it, accuracy_at_iteration[1][it][0], accuracy_at_iteration[2][it][0], accuracy_at_iteration[3][it][0], accuracy_at_iteration[4][it][0])

print(" ", "1", "2", "3", "4")
for it in [100, 200, 400, 800, 1600, 3200, 6400]:
    print(it, accuracy_at_iteration[1][it][1], accuracy_at_iteration[2][it][1], accuracy_at_iteration[3][it][1], accuracy_at_iteration[4][it][1])


  1 2 3 4
100 0.78125 0.02564102564102564 0.0 0.0
200 0.78125 0.1282051282051282 0.025 0.0
400 0.78125 0.07692307692307693 0.0 0.0
800 0.78125 0.1794871794871795 0.0 0.0
1600 0.78125 0.3333333333333333 0.025 0.0
3200 0.78125 0.358974358974359 0.0 0.0
6400 0.78125 0.3333333333333333 0.0 0.0
  1 2 3 4
100 0.9375 0.6153846153846154 0.15 0.05
200 0.9375 0.6666666666666666 0.275 0.05
400 0.9375 0.5897435897435898 0.2 0.125
800 0.9375 0.7692307692307693 0.25 0.1
1600 0.9375 0.7948717948717948 0.325 0.15
3200 0.9375 0.7435897435897436 0.4 0.2
6400 0.9375 0.717948717948718 0.325 0.15


In [48]:
print("""1    2   3   4 
0.9375  0.38461538461538464  0.125  0.05
1    2   3   4 
0.96875  0.6923076923076923  0.4  0.275""")

1    2   3   4 
0.9375  0.38461538461538464  0.125  0.05
1    2   3   4 
0.96875  0.6923076923076923  0.4  0.275
